In [ ]:
pip install --force-reinstall "torch==2.8.0" "torchvision==0.23.0" "torchaudio==2.8.0" "transformers>=4.44.0,<4.45" "accelerate>=0.34.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 726.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193

In [ ]:
pip install datasets

In [ ]:
!pip install -q bertopic umap-learn hdbscan

In [ ]:
import json
import pandas as pd
from pathlib import Path
import re
from transformers import pipeline
from datasets import load_dataset
from datasets import Dataset
from tqdm.auto import tqdm
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from hdbscan import HDBSCAN
import numpy as np
import pandas as pd
import numpy as np
import re

/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.


In [ ]:
ds = load_dataset("ShivomH/Mental-Health-Conversations")
df_raw = pd.DataFrame(ds["train"])

records = []

for idx, conversation in enumerate(df_raw["messages"]):
    if isinstance(conversation, list):
        for turn, msg in enumerate(conversation, start=1):
            if isinstance(msg, dict):
                records.append({
                    "conversation_id": f"conv_{idx+1}",
                    "turn": turn,
                    "role": msg.get("role"),
                    "text": msg.get("content")
                })

df_flat = pd.DataFrame(records)
df_flat = df_flat[df_flat["role"] != "system"].reset_index(drop=True)

README.md:   0%|          | 0.00/469 [00:00<?, ?B/s]

Mental_Health_MultiTurn.jsonl:   0%|          | 0.00/474M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/99086 [00:00<?, ? examples/s]

In [ ]:
df_flat = df_flat[~df_flat["text"].str.startswith("[")].reset_index(drop=True)

# convert df_flat to a Hugging Face Dataset (faster streaming to GPU)
dataset = Dataset.from_pandas(df_flat)

# initialize model on GPU
emo = pipeline(
    "text-classification",
    model="joeddav/distilbert-base-uncased-go-emotions-student",
    top_k=1,            # only top emotion instead of all 28
    device=-1,
    batch_size=8       # efficient GPU batches
)

NEUTRAL_BAND = 0.40
POS = {"admiration","amusement","approval","desire","excitement",
       "gratitude","joy","love","optimism","pride","relief"}
NEG = {"anger","annoyance","disappointment","disapproval",
       "embarrassment","fear","grief","remorse","sadness","disgust","nervousness","loneliness"}

# run model directly on dataset (no manual loop)
texts = list(dataset["text"][:8000])
results = list(tqdm(emo(texts, batch_size=8), total=len(texts)))

# extract top label + score
fine_labels = [r[0]["label"].lower() for r in results]
scores = [float(r[0]["score"]) for r in results]

# map to coarse sentiment
def to_sentiment(lbl, sc):
    if sc < NEUTRAL_BAND:
        return "neutral"
    elif lbl in POS:
        return "positive"
    elif lbl in NEG:
        return "negative"
    else:
        return "neutral"

labels = [to_sentiment(l, s) for l, s in zip(fine_labels, scores)]

# build labeled dataframe
df1 = df_flat.head(8000).copy()
df1["emotion_label"] = fine_labels
df1["sentiment_label"] = labels
df1["sentiment_score"] = scores

df1.head()

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/421 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

  0%|          | 0/8000 [00:00<?, ?it/s]

,conversation_id,turn,role,text,emotion_label,sentiment_label,sentiment_score
0,conv_1,2,user,I've been feeling so sad and overwhelmed latel...,sadness,neutral,0.342626
1,conv_1,3,assistant,"Hey there, I'm here to listen and support you....",caring,neutral,0.192634
2,conv_1,4,user,"I recently got a promotion at work, which I th...",realization,neutral,0.139096
3,conv_1,5,assistant,I can understand how it can be overwhelming wh...,sadness,neutral,0.299411
4,conv_1,6,user,"Well, the workload has increased significantly...",disappointment,neutral,0.141864


In [ ]:
# --- Aggregate all messages within each conversation ---
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from sklearn.feature_extraction.text import CountVectorizer
from hdbscan import HDBSCAN
import pandas as pd

# Combine messages per conversation
df_conv = (
    df1.groupby("conversation_id")["text"]
       .apply(lambda x: " ".join(x))
       .reset_index()
)
print(f"Number of conversations: {len(df_conv)}")

# Prepare vectorizer, clustering, and representation model
vectorizer_model = CountVectorizer(stop_words="english", min_df=2)
custom_hdbscan = HDBSCAN(
    min_cluster_size=2,   # smaller clusters allowed
    min_samples=1,        # less conservative
    cluster_selection_epsilon=0.1,  # encourages splitting
    metric="euclidean"
)
representation_model = KeyBERTInspired()

# Create BERTopic model (uses default dimensionality reduction)
topic_model = BERTopic(
    embedding_model="sentence-transformers/all-roberta-large-v1",
    vectorizer_model=vectorizer_model,
    hdbscan_model=custom_hdbscan,
    representation_model=representation_model,
    nr_topics=None,       # disable auto-merge
    calculate_probabilities=False,
    verbose=True
)

# Fit model on conversation-level text
texts = df_conv["text"].astype(str).tolist()
topics, probs = topic_model.fit_transform(texts)

# Retrieve and clean topic info
topic_info = topic_model.get_topic_info()

def clean_name(name):
    if name.startswith("-1"):
        return "General / Mixed"
    words = [w for w in name.split("_") if len(w) > 3]
    return " ".join(words[:3]).capitalize() if words else "General"

topic_info["clean_name"] = topic_info["Name"].apply(clean_name)

# Map topics back to df_conv
df_conv["theme_topic_id"] = topics
df_conv["themes"] = df_conv["theme_topic_id"].map(
    topic_info.set_index("Topic")["clean_name"]
).fillna("Other")

# Overview
print("\n=== TOP THEMES ===")
print(topic_info[["Topic", "clean_name", "Count"]].head(10))


2025-10-28 20:01:35,450 - BERTopic - Embedding - Transforming documents to embeddings.


Number of conversations: 491


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2025-10-28 20:02:14,872 - BERTopic - Embedding - Completed ✓
2025-10-28 20:02:14,873 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-10-28 20:02:23,635 - BERTopic - Dimensionality - Completed ✓
2025-10-28 20:02:23,637 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-10-28 20:02:23,695 - BERTopic - Cluster - Completed ✓
2025-10-28 20:02:23,701 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-10-28 20:02:29,263 - BERTopic - Representation - Completed ✓



=== TOP THEMES ===
   Topic                      clean_name  Count
0     -1                 General / Mixed     57
1      0           Coping feel emotional     11
2      1   Depression counseling improve      9
3      2         Feel feeling expressing      9
4      3             Anxiety feel coping      9
5      4  Stress mindfulness overwhelmed      8
6      5     Overwhelmed grieve emotions      8
7      6       Emotions coping emotional      8
8      7         Depression anxiety feel      8
9      8       Feel feels responsibility      7


In [ ]:
import pandas as pd
import numpy as np

# --- Conversational Metrics for df1 ---
# Assumes df1 has: 'text', 'conversation_id', and 'role' ('user' vs. 'assistant')

df1 = df1.copy()

# number of words in each message
df1["message_length"] = df1["text"].astype(str).apply(lambda x: len(x.split()))

# does the message end with a question mark?
df1["is_question"] = df1["text"].astype(str).str.strip().str.endswith("?").astype(int)



In [ ]:
import pandas as pd
import numpy as np

# ISSUE DOMAIN — Detects main life area based on keywords
def detect_issue_domain(text):
    text_lower = text.lower()
    domains = {
        "work": ["work", "job", "career", "boss", "office", "manager", "deadline"],
        "relationships": ["relationship", "partner", "boyfriend", "girlfriend", "marriage", "love", "breakup", "gf", "bf"],
        "family": ["family", "parent", "mother", "father", "sibling", "child", "kids", "home"],
        "health": ["health", "pain", "sick", "doctor", "therapy", "body", "tired", "sleep"],
        "self_growth": ["goal", "purpose", "confidence", "motivation", "improve", "learn", "focus"]
    }
    for domain, words in domains.items():
        if any(w in text_lower for w in words):
            return domain
    return "other"

df_conv["issue_domain"] = df_conv["text"].apply(detect_issue_domain)

# TURN COUNT & USER TALK RATIO
conv_counts = (
    df1.groupby(["conversation_id", "role"])["text"]
       .count()
       .unstack(fill_value=0)
       .reset_index()
)
conv_counts["turn_count"] = conv_counts.drop(columns=["conversation_id"]).sum(axis=1)
conv_counts["user_talk_ratio"] = conv_counts.get("user", 0) / conv_counts["turn_count"]

df_conv = df_conv.merge(conv_counts[["conversation_id", "turn_count", "user_talk_ratio"]],
                        on="conversation_id", how="left")

# SATISFACTION SIGNAL
gratitude_keywords = [
    "thank", "thanks", "appreciate", "helped", "that helps", "feel better",
    "makes sense", "feel calmer", "that’s useful", "good advice", "understand better"
]

def detect_satisfaction_signal(text):
    text_lower = text.lower()
    return any(kw in text_lower for kw in gratitude_keywords)

satisfaction = (
    df1[df1["role"] == "user"]
    .groupby("conversation_id")["text"]
    .apply(lambda msgs: any(detect_satisfaction_signal(t) for t in msgs))
    .reset_index()
    .rename(columns={"text": "satisfaction_signal"})
)
df_conv = df_conv.merge(satisfaction, on="conversation_id", how="left")

# DROP-OFF POINT — Last user turn
dropoff_df = (
    df1[df1["role"] == "user"]
    .groupby("conversation_id")["turn"]
    .max()
    .reset_index()
    .rename(columns={"turn": "drop_off_point"})
)
df_conv = df_conv.merge(dropoff_df, on="conversation_id", how="left")

# Merge conversation themes back into df1
df1 = df1.merge(df_conv[["conversation_id", "themes"]],
                on="conversation_id", how="left")

# Define helper for transition counting
def count_transitions(labels):
    """Count how many times a sequence changes value (ignoring NaN)."""
    labels = [x for x in labels if pd.notna(x)]
    if len(labels) <= 1:
        return 0
    return sum(labels[i] != labels[i - 1] for i in range(1, len(labels)))

# EMOTION & SENTIMENT TRANSITIONS
print("Computing emotion/sentiment transitions...")

emotion_transitions = (
    df1.groupby("conversation_id")["emotion_label"]
       .apply(list)
       .apply(count_transitions)
       .reset_index()
       .rename(columns={"emotion_label": "emotion_transition_count"})
)

sentiment_transitions = (
    df1.groupby("conversation_id")["sentiment_label"]
       .apply(list)
       .apply(count_transitions)
       .reset_index()
       .rename(columns={"sentiment_label": "sentiment_transition_count"})
)

df_conv = (
    df_conv.merge(emotion_transitions, on="conversation_id", how="left")
           .merge(sentiment_transitions, on="conversation_id", how="left")
)

# Normalize transition counts per conversation length
if "turn_count" in df_conv.columns:
    df_conv["emotion_transition_rate"] = (
        df_conv["emotion_transition_count"] / df_conv["turn_count"].replace(0, np.nan)
    )
    df_conv["sentiment_transition_rate"] = (
        df_conv["sentiment_transition_count"] / df_conv["turn_count"].replace(0, np.nan)
    )
else:
    print("⚠️ turn_count not found — skipping normalization.")
    df_conv["emotion_transition_rate"] = None
    df_conv["sentiment_transition_rate"] = None

# COMPLETION OUTCOME
def infer_outcome(sub_df):
    user_msgs = sub_df[sub_df["role"] == "user"]
    if user_msgs.empty:
        return "unknown"
    last_user = user_msgs.iloc[-1]
    if last_user.get("sentiment_label") == "negative":
        return "abandoned"
    elif any(detect_satisfaction_signal(t) for t in user_msgs["text"].tolist()):
        return "resolved"
    else:
        return "ongoing"

completion_outcomes = (
    df1.groupby("conversation_id")
       .apply(infer_outcome)
       .reset_index(name="completion_outcome")
)
df_conv = df_conv.merge(completion_outcomes, on="conversation_id", how="left")

# SENTIMENT START → END SHIFT (improvement trend)
def detect_shift(series):
    series = [x for x in series if pd.notna(x)]
    if len(series) < 2:
        return ("none", "none", 0)
    start, end = series[0], series[-1]
    # sentiment trend: +1 if improving, -1 if worsening
    if start == end:
        trend = 0
    elif (start == "negative" and end in ["neutral", "positive"]) or (start == "neutral" and end == "positive"):
        trend = +1
    else:
        trend = -1
    return (start, end, trend)

sentiment_shift = (
    df1.groupby("conversation_id")["sentiment_label"]
       .apply(detect_shift)
       .apply(pd.Series)
       .reset_index()
       .rename(columns={0: "sentiment_start", 1: "sentiment_end", 2: "sentiment_trend"})
)
df_conv = df_conv.merge(sentiment_shift, on="conversation_id", how="left")


Computing emotion/sentiment transitions...


/tmp/ipython-input-410393232.py:127: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(infer_outcome)


In [ ]:
from transformers import pipeline
from tqdm.auto import tqdm
import re

# Define sensitive categories
sensitive_labels = [
    "suicidal thoughts or self-harm",
    "death or grief",
    "violence or abuse",
    "trauma or assault",
    "mental illness or severe anxiety",
    "non-sensitive / general"
]

# Initialize model
clf = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli",
    device=-1  # use 0 if you have GPU
)

# Quick prefilter: run full model only on likely-risk messages
risk_keywords = [
    "suicide", "kill myself", "want to die", "end my life", "die",
    "cut myself", "hurt myself", "self harm", "death", "dead", "funeral",
    "grief", "rape", "abuse", "violence", "attack", "trauma", "panic",
    "anxiety", "depression", "bipolar", "schizo"
]

def is_potentially_sensitive(text):
    text_lower = text.lower()
    return any(kw in text_lower for kw in risk_keywords)

df1["potentially_sensitive"] = df1["text"].apply(is_potentially_sensitive)

def classify_sensitive(text):
    text = text.strip()
    if not text:
        return {"label": "none", "score": 0.0}
    result = clf(text, candidate_labels=sensitive_labels, multi_label=True)
    scores = dict(zip(result["labels"], result["scores"]))
    # keep any label > 0.5
    high_conf = {k: v for k, v in scores.items() if v >= 0.5}
    if not high_conf:
        return {"label": "none", "score": max(scores.values())}
    top = max(high_conf, key=high_conf.get)
    return {"label": top, "score": high_conf[top]}

# Run on a subset (saves time)
sensitive_subset = df1[df1["potentially_sensitive"]].copy()

results = [classify_sensitive(t) for t in tqdm(sensitive_subset["text"].tolist())]

sensitive_subset["sensitive_label"] = [r["label"] for r in results]
sensitive_subset["sensitive_score"] = [r["score"] for r in results]
sensitive_subset["is_sensitive"] = sensitive_subset["sensitive_label"].apply(lambda x: x != "none")

# Merge predictions back to full dataset
df1 = df1.merge(
    sensitive_subset[["conversation_id", "turn", "sensitive_label", "sensitive_score", "is_sensitive"]],
    on=["conversation_id", "turn"],
    how="left"
)

# Fill unprocessed messages with neutral defaults
df1["sensitive_label"] = df1["sensitive_label"].fillna("none")
df1["sensitive_score"] = df1["sensitive_score"].fillna(0.0)
df1["is_sensitive"] = df1["is_sensitive"].fillna(False)

# Conversation-level sensitivity: if any message in conversation is sensitive
conv_sensitivity = (
    df1.groupby("conversation_id")[["is_sensitive", "sensitive_label"]]
    .agg({
        "is_sensitive": "any",
        "sensitive_label": lambda x: ", ".join(set(x) - {"none"})
    })
    .reset_index()
)

df_conv = df_conv.merge(conv_sensitivity, on="conversation_id", how="left")
df_conv["sensitive_label"] = df_conv["sensitive_label"].replace("", "none")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

  0%|          | 0/1052 [00:00<?, ?it/s]

In [ ]:
df1.head(5)

,conversation_id,turn,role,text,emotion_label,sentiment_label,sentiment_score,message_length,is_question,themes,potentially_sensitive,sensitive_label,sensitive_score,is_sensitive
0,conv_1,2,user,I've been feeling so sad and overwhelmed latel...,sadness,neutral,0.342626,19,0,Depressed distress feel,False,none,0.0,False
1,conv_1,3,assistant,"Hey there, I'm here to listen and support you....",caring,neutral,0.192634,28,1,Depressed distress feel,False,none,0.0,False
2,conv_1,4,user,"I recently got a promotion at work, which I th...",realization,neutral,0.139096,36,0,Depressed distress feel,False,none,0.0,False
3,conv_1,5,assistant,I can understand how it can be overwhelming wh...,sadness,neutral,0.299411,46,1,Depressed distress feel,False,none,0.0,False
4,conv_1,6,user,"Well, the workload has increased significantly...",disappointment,neutral,0.141864,48,0,Depressed distress feel,False,none,0.0,False


In [ ]:
df_conv.head(5)

,conversation_id,text,theme_topic_id,themes,issue_domain,turn_count,user_talk_ratio,satisfaction_signal,drop_off_point,emotion_transition_count,sentiment_transition_count,emotion_transition_rate,sentiment_transition_rate,completion_outcome,sentiment_start,sentiment_end,sentiment_trend,is_sensitive,sensitive_label
0,conv_1,I've been feeling so sad and overwhelmed latel...,17,Depressed distress feel,work,12,0.5,True,12,11,2,0.916667,0.166667,resolved,neutral,neutral,0,False,none
1,conv_10,I'm feeling so embarrassed and overwhelmed. It...,-1,General / Mixed,work,14,0.5,False,14,12,6,0.857143,0.428571,ongoing,neutral,neutral,0,False,none
2,conv_100,"Hi. I've been feeling a bit nervous lately, es...",-1,General / Mixed,work,18,0.5,False,18,12,3,0.666667,0.166667,ongoing,negative,neutral,1,False,none
3,conv_101,"I've been feeling so hopeful lately, but at th...",54,Counseling stress feeling,work,12,0.5,True,12,8,2,0.666667,0.166667,resolved,neutral,neutral,0,False,none
4,conv_102,I'm feeling so confused lately. My self-esteem...,86,Esteem expectations improving,work,16,0.5,True,16,11,2,0.687500,0.125000,resolved,neutral,neutral,0,False,none


In [ ]:
df1.to_csv("individual_text.csv")

In [ ]:
df_conv.to_csv("conversation.csv")

In [7]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download resources once
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")

df_text = pd.read_csv("/individual_text.csv")

# Base stopwords
stop_words = set(stopwords.words("english"))

# Add conversational fillers & weak words
extra_stops = {
    "like", "yeah", "okay", "ok", "still", "lot", "really", "something",
    "anything", "everything", "nothing", "get", "got", "going", "thing",
    "things", "maybe", "actually", "oh", "yes", "no", "right", "one", "think",
    "see", "sure", "time", "much", "make", "even", "say", "said", "know",
    "feel", "felt", "just", "also", "well", "way", "could", "would", "might"
}
stop_words = stop_words.union(extra_stops)

# Contraction expansion
contraction_patterns = {
    r"\bI'm\b": "I am",
    r"\bI've\b": "I have",
    r"\bI'll\b": "I will",
    r"\bI'd\b": "I would",
    r"\byou're\b": "you are",
    r"\byou've\b": "you have",
    r"\bhe's\b": "he is",
    r"\bshe's\b": "she is",
    r"\bit's\b": "it is",
    r"\bwe're\b": "we are",
    r"\bthey're\b": "they are",
    r"\bcan't\b": "cannot",
    r"\bwon't\b": "will not",
    r"\bdon't\b": "do not",
    r"\bdoesn't\b": "does not",
    r"\bdidn't\b": "did not",
    r"\bshouldn't\b": "should not",
    r"\bwouldn't\b": "would not",
    r"\baren't\b": "are not",
    r"\bwasn't\b": "was not",
    r"\bweren't\b": "were not",
    r"\bain't\b": "is not",
    r"’": "'",  # normalize curly quotes
}

lemmatizer = WordNetLemmatizer()

def clean_text_for_wordcloud(text):
    if not isinstance(text, str):
        return ""
    text = text.lower().strip()

    # Expand contractions
    for pattern, repl in contraction_patterns.items():
        text = re.sub(pattern, repl, text)

    # Remove punctuation and numbers
    text = re.sub(r"[^a-z\s]", " ", text)

    # Tokenize
    tokens = text.split()

    # Remove stopwords and short words, lemmatize
    cleaned = [
        lemmatizer.lemmatize(w)
        for w in tokens
        if w not in stop_words and len(w) > 2
    ]
    return " ".join(cleaned)

# Apply cleaning
df_text["text_clean_wc"] = df_text["text"].apply(clean_text_for_wordcloud)

# Preview
print(df_text[["text", "text_clean_wc"]].head(10))

# Save ready for word cloud
df_text.to_csv("individual_text_wordcloud.csv", index=False)
print("✅ Saved → individual_text_wordcloud.csv (ready for word cloud)")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


                                                text  \
0  I've been feeling so sad and overwhelmed latel...   
1  Hey there, I'm here to listen and support you....   
2  I recently got a promotion at work, which I th...   
3  I can understand how it can be overwhelming wh...   
4  Well, the workload has increased significantly...   
5  It sounds like you're dealing with a lot of pr...   
6  I've been trying to prioritize my tasks and de...   
7  It's great to hear that you're already impleme...   
8  You're right. I haven't really opened up about...   
9  It's completely normal to feel that way, but r...   

                                       text_clean_wc  
0  feeling sad overwhelmed lately work become mas...  
1  hey listen support sound work challenging late...  
2  recently promotion work thought exciting added...  
3  understand overwhelming faced higher expectati...  
4  workload increased significantly find hard mai...  
5  sound dealing pressure perform succeed remembe... 